# <span style="color:darkblue">05-Plotting</span>

In this notebook, we will create plots for all the conditions and RNAs in our dataset. Some of these figure were also used in the publication linked to this dataset. 

## 5.1 - Load libraries

In [ ]:
from glob import glob
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [ ]:
# adjust plotting settings
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 18}

matplotlib.rc('font', **font)

In [ ]:
# paths
root_dir = '../data/zipped_example_data_smFISH_C.albicans/'

In [ ]:
files = glob(f"{root_dir}/Results/*.csv")

dfs = []
for file in files:
    print(f'loading {file} ...')
    filename=Path(file).stem
    df = pd.read_csv(file)
    df['strain'], df['mRNA'], df['condition'], df['channel'], df['fov'] = filename.split('_')
    dfs.append(df)
    print('... done')
df = pd.concat(dfs).reset_index()

In [ ]:
# inspect data

df.head()

In [ ]:
# store variables:

mRNAs = df['mRNA'].unique()
conditions = df['condition'].unique()

***

## 5.1 - Histograms of mRNA counts and eccentricity

In [ ]:
nrows = len(mRNAs)
ncols = len(conditions)

figure, axes= plt.subplots(nrows=nrows, ncols=ncols, figsize=(10,8))
axes = np.array(axes).reshape(nrows, ncols)

for row, mRNA in enumerate(mRNAs):
    for col, condition in enumerate(conditions):
        
        sns.histplot(
            data=df[(df['mRNA']==mRNA) & (df['condition']==condition)], x='total_RNAs_per_cell',ax=axes[row, col],binwidth=1, palette='viridis')
        
        axes[row,col].set_xlim(0,90)
        axes[row,col].set_ylim(0,80)
        
        axes[row,col].set_title(f'expression of {mRNA.split("Q")[0]} in {condition}')
        
plt.tight_layout()

In [ ]:
figure, ax= plt.subplots(figsize=(10,5))
ax=sns.histplot(data=df[(df['mRNA']=='CLB2Q670')], x='total_RNAs_per_cell'
                ,hue='condition',bins=50,ax=ax,palette='deep')
ax.set_title('CLB2' )

#plt.savefig('FV-CLB2Q670_histplot.svg')

In [ ]:
figure,ax= plt.subplots(figsize=(10,5))
ax=sns.histplot(
    data=df[(df['mRNA']=='CLB2Q670')], x='total_RNAs_per_cell',hue='condition',bins=50,ax=ax, 
    element="poly",palette='deep')
ax.set_title('CLB2')

#plt.savefig('FV_CLB2Q670_polyplot.svg')

In [ ]:
figure, ax= plt.subplots(figsize=(10,5))
ax=sns.histplot(data=df[(df['mRNA']=='EFG1Q670')],
                x='total_RNAs_per_cell',hue='condition',bins=50,ax=ax, palette='deep')
ax.set_title('EFG1')

#plt.savefig('FV_EFG1Q670_histplot.svg')

In [ ]:
figure,ax= plt.subplots(figsize=(10,5))
ax=sns.histplot(data=df[(df['mRNA']=='EFG1Q670')], x='total_RNAs_per_cell',hue='condition',bins=50,ax=ax, element="poly",palette='deep')
ax.set_title('EFG1' )

#plt.savefig('FV_EFG1Q670_polyplot.svg')

In [ ]:
figure,ax= plt.subplots(figsize=(10,5))
ax=sns.histplot(data=df, x='eccentricity',hue='condition',bins=100,ax=ax, palette='deep')
ax.set_title('Eccentricity' )

#plt.savefig('FV_Eccenticity_histplot.svg')

***

## 5.2 - KDEplots mRNA vs eccentricity

In [ ]:
nrows = len(mRNAs)
ncols = len(conditions)

figure, axes= plt.subplots(nrows=nrows, ncols=ncols, figsize=(10,8))
axes = np.array(axes).reshape(nrows, ncols)

for row, mRNA in enumerate(mRNAs):
    for col, condition in enumerate(conditions):
        sns.scatterplot(data=df[(df['mRNA']==mRNA) & (df['condition']==condition)], x='total_RNAs_per_cell',y='eccentricity',ax=axes[row,col],palette='viridis')
        sns.kdeplot(data=df[(df['mRNA']==mRNA) & (df['condition']==condition)],x='total_RNAs_per_cell',y='eccentricity', levels=5, color="b", linewidths=1,ax=axes[row, col])
        axes[row,col].set_xlim(0,80)
        axes[row,col].set_ylim(0,1)
        
        axes[row,col].set_title(f'expression of {mRNA.split("Q")[0]} in {condition}')
        
#plt.savefig(f'scatter_kdeplot_{mRNA}_{condition}.svg')

plt.tight_layout()

In [ ]:
nrows = len(mRNAs)
ncols = len(conditions)

axes = np.array(axes).reshape(nrows, ncols)
for row, mRNA in enumerate(mRNAs):
    for col, condition in enumerate(conditions):

        g=sns.jointplot(data=df[(df['mRNA']==mRNA) & (df['condition']==condition)],x='total_RNAs_per_cell',y='eccentricity',palette='viridis', color="b", linewidths=1,hue="mRNA")
        g.ax_joint.set_yticks([x/10 for x in range(0,12,2)])
        plt.suptitle(f'{condition}_{mRNA}')
        plt.tight_layout()
        #plt.savefig(f'fv_jointplot_{mRNA}_{condition}.svg')

***

## 5.3 - Stripplots

In [ ]:
ax = sns.stripplot(
    data=df,
    x="mRNA", 
    y="spots_per_cell", 
    orient="v",palette='viridis' ,hue='condition',dodge=True, s=3, alpha=0.3)

ax.set_title('spots per cell')
ax.set_ylim(0,85)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

#plt.savefig('stripplot_spot_per_cell.svg')

### <span style="color:seagreen"> 5.3.1 - Plotting only expressing cells </span>

In [ ]:
ax=sns.stripplot(
    data=df[df['total_RNAs_per_cell']>0],
    x="condition", y="total_RNAs_per_cell", orient="v",palette='viridis' ,hue='mRNA',dodge=True, s=3, alpha=0.5
)
ax.set_title('RNAs_per_cell')
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
ax.set_ylim(0,85)

# plot the mean line
sns.boxplot(showmeans=True,
            meanline=True,
            meanprops={'color': 'k', 'ls': '-', 'lw': 1},
            medianprops={'visible': False},
            whiskerprops={'visible': False},
            zorder=10,
            x="condition",
            y="total_RNAs_per_cell",
            hue='mRNA',
            data=df[df['total_RNAs_per_cell']>0],
            showfliers=False,
            showbox=False,
            showcaps=False)
plt.legend(title='mRNA', loc='upper right', labels=['CLB2Q670', 'EFG1Q670'],markerscale=5)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

#plt.savefig('FV_stripplot_total_RNAs.svg')

### <span style="color:seagreen"> 5.3.2 - Plotting nascent mRNA per cell </span>

In [ ]:
ax=sns.stripplot(
    data=df[df['nascent_RNAs']>0],
    x="condition", y="nascent_RNAs", orient="v",palette='viridis' ,hue='mRNA',dodge=True, s=3, alpha=0.5
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

ax.set_title('nascent RNAs per cell')
ax.set_ylim(0,35)

# plot the mean line
sns.boxplot(showmeans=True,
            meanline=True,
            meanprops={'color': 'k', 'ls': '-', 'lw': 1},
            medianprops={'visible': False},
            whiskerprops={'visible': False},
            zorder=10,
            x="condition",
            y="nascent_RNAs",
            hue='mRNA',
            data=df[df['nascent_RNAs']>0],
            showfliers=False,
            showbox=False,
            showcaps=False)
plt.legend(title='mRNA', loc='upper right', labels=['CLB2Q670', 'EFG1Q670'],markerscale=5)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

#plt.savefig('FV_nascent_stripplot.svg')